## Gaussian Process Regression Model

#### Adding the libraries

In [2]:
import pandas as pd
import matplotlib as plt
import ast
import numpy as np
import seaborn as sns
from statistics import mean

#### Add the dataset

In [3]:
dataset = pd.read_pickle('../Data/features_2001.pkl')
dataset.head()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80237 entries, 0 to 80236
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            80237 non-null  int64  
 1   diversity     80237 non-null  float64
 2   venue_rank    80237 non-null  float64
 3   venue_MPI     80237 non-null  int64  
 4   venue_TPI     80237 non-null  int64  
 5   productivity  80237 non-null  float64
 6   H_index       80237 non-null  float64
 7   author_rank   80237 non-null  float64
 8   author_MPI    80237 non-null  float64
 9   author_TPI    80237 non-null  float64
 10  versatility   80237 non-null  float64
 11  n_citation    80237 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 8.0 MB


#### The model 

In [4]:
#Import all gaussian Processes libraries 
import sklearn.gaussian_process as gp
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.model_selection import train_test_split


In [17]:
# define dataset: creates clusters/ list of lists that contain all feature values except n-citatons
X = dataset.iloc[1:15000, 1:11].values 
# the result(n_citation), list of n-citations, assuming n-citations is in 13th column
y = dataset.iloc[1:15000, 11].values

In [18]:
#Split into observtions(X) and results(y) into 80/20 training/test split
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.20, random_state = 0)


In [19]:
#Create gaussian model kernel with the mean(constant at 1) and RBF with lenghscale = 10.0 with lenghscale bounds of (0.001, 1000.0)

##Different covariance kernels being setup with thier respective parameters, Can be selected one at a time

#kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))

#When nu = 0.5 , the Matérn kernel becomes identical to the absolute exponential kernel
#kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.Matern(1.0, (1e-3, 1e3), nu = 0.5)

#When nu = 1.5 then the exponential function becomes once differentiable 
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.Matern(1.0, (1e-3, 1e3), nu = 1.5)



In [20]:
#When nu = 2.5 then the exponential function becomes twice differentiable 
#kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.Matern(1.0, (1e-3, 1e3), nu = 2.5)

#kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RationalQuadratic(length_scale=1.0, alpha=1.0, length_scale_bounds=(1e-05, 100000.0), alpha_bounds=(1e-05, 100000.0))

# Where Alpha is variance of independently, identically distributed Gaussian noise form labels(y)
#Where normalize_y refers to constant mean function:either zero if False or the training data mean if True
#Where n_restarts_optimizer is multiple restarts of the optimizer with different initializations is used if log log marginal likelihood may not be convex
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=False)

model.fit(X_train, y_train)
#get hyper parameters of kernel 




GaussianProcessRegressor(alpha=0.1,
                         kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=10)

In [15]:
y_predict, std = model.predict(X_test, return_std=True)

params = model.kernel_.get_params()
params
#y_predict
#std
#MSE
MSE = ((y_predict-y_test)**2).mean()
#MSE
print(MSE)

2699.848790166394


In [16]:
correlation_matrix = np.corrcoef(y_predict, y_test)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

print(r_squared)

0.06164771866634619
